In [ ]:
# Initialize OK
from client.api.notebook import Notebook
ok = Notebook('hw5.ok')

# Homework 5: Exploring bias through Cook County’s property assessments

## Due Date: 11:59pm Monday, March 30

### Collaboration Policy

Data science is a collaborative activity. While you may talk with others about the homework, we ask that you **write your solutions individually**. If you do discuss the assignments with others please **include their names** in the collaborators cell below.

**Collaborators:** *list names here*

## Introduction

In this homework, we will go through the iterative process of specifying, fitting, and analyzing the performance of a  model.  

In the first portion of the assignment, we will guide you through some basic exploratory data analysis (EDA), laying out the thought process that leads to certain modeling decisions. Next, you will add a new feature to the dataset, before specifying and fitting a linear model to a few features of the housing data to predict housing prices. Finally, we will analyze the error of the model and brainstorm ways to improve the model's performance.

After this homework, you should feel comfortable with the following:

1. Simple feature engineering
1. Using sklearn to build linear models
1. Building a data pipeline using pandas

Next week's homework will continue working with this dataset to address more advanced and subtle issues with modeling.

## HCE Learning Outcomes

By working through this homework, students will be able to:
* Identify sources of bias within social and technical decisions. 
* Understand how bias emerges from the human contexts of data science work, specifically through professions and institutions.
* Recognize that, because of this human context, bias is structural to data science throughout the data lifecycle rather than an individual, subjective variable that can be eradicated.
* Analyze the effects of both deliberate and unintentional choices made throughout their work in order to meaningfully address questions of fairness.


## Score Breakdown

*To be edited by course staff.*

In [1]:
import numpy as np

import pandas as pd
from pandas.api.types import CategoricalDtype

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Plot settings
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

# Part 0: Contextualizing the Data

### The Cook County Assessor’s Office

The dataset you’ll be working with comes from the Cook County Assessor’s Office (CCAO) in Illinois, a government institution that determines property taxes across most of Chicago’s metropolitan area and its nearby suburbs. These property tax assessments are based on property values estimated using statistical models that consider multiple factors, such as real estate value and construction cost.

This system, however, is not without flaws. In late 2017, a lawsuit was filed against the office of Cook County Assessor Joseph Berrios for producing [“racially discriminatory assessments and taxes."](https://harris.uchicago.edu/news-events/news/prof-chris-berry-testifies-institutional-racism-cook-county-property-taxes) The lawsuit included claims that the assessor’s office undervalued high-priced homes and overvalued low-priced homes, creating a visible divide along racial lines: Wealthy homeowners, who were typically white, [paid less in property taxes](https://www.clccrul.org/bpnc-v-berrios-facts?rq=berrios), whereas working-class, [non-white homeowners paid more](https://www.chicagotribune.com/news/breaking/ct-cook-county-assessor-berrios-sued-met-20171214-story.html).

Although there are certainly many layers to the discrimination in this case (Chicago’s history of redlining, for example), unfair property taxation arose in part because of the tax lawyer industry. After the assessor’s office releases their property valuations, tax lawyers have the power to negotiate for lower valuations - hence, lower property taxes - in an appeals process. And because hiring a tax lawyer is limited to those who can afford it, wealthy property owners ultimately paid a disproportionate amount of tax compared to those of lower socioeconomic status - people who are predominantly from Hispanic and Black communities in Cook County. In this way, inequity along racial lines is systemically built into the property assessment system.

### Open Data and the CCAO

To push back against this systemic inequity, Berrios’ successor, Cook County Assessor Fritz Kaegi, promised to increase transparency in the assessment process. The hope was that, by exposing the inner workings of the CCAO’s property valuation process, their assessment results could be publicly verified as accurate and therefore trusted to be fair. Additionally, tax lawyers would be unable to contest the validity of the Office’s data and modeling process. 

Through Cook County’s [open data hub](https://datacatalog.cookcountyil.gov/), the CCAO publishes the data they use in their model to valuate properties. Their [model, written in R,](https://gitlab.com/ccao-data-science---modeling) is also available for viewing. If you’re interested in reading more about how this initiative is being presented by the CCAO, you can check out their [Medium article](https://medium.com/@AssessorCook/why-the-cook-county-assessors-office-made-its-residential-assessment-code-and-data-public-c964acfa7b0f).

Though the CCAO’s transparency initiative is the first of its kind in the country, it is nonetheless open to critique: The Office itself solicits community input on the quality and fairness of its model. And in this homework, we will address the different types of bias that may arise in the modeling process.

## HCE: Question -1

Based on the preceding introductory paragraphs, in what way was the previous property assessment process discriminatory? How has the CCAO decided to combat this issue? (1-2 sentences total)


`TODO`: *Write your answer here*

### The Data 

The Cook County dataset you'll be using in this homework contains residential sales data that the CCAO uses to assess property values. A more detailed description of each variable is available on [their website](https://datacatalog.cookcountyil.gov/Property-Taxation/Cook-County-Assessor-s-Residential-Sales-Data/5pge-nu6u). There, the list of column values notes what each entry represents and sometimes cautions about the quality of a given variable. **You should take some time to familiarize yourself with the variables before moving forward.**

You can also read more about the context of this dataset from the [Cook County Assessor's Office Data Narrative](https://datacatalog.cookcountyil.gov/stories/s/Cook-County-Assessor-Valuation-Data-Release/p2kt-hk36).

In [ ]:
# Display a list of the columns in the dataset.

You might notice that the columns contain references to apartments, condos, and commercial buildings. However, we’ll only be focusing on single-family houses for this homework. Run the following cell to filter out entities that aren’t homes.

In [ ]:
# Filter out properties that aren't single-family homes. Refer to the 'Technical Checks' Google Doc.

The data is split into training and test sets with X and Y observations, respectively.

In [2]:
training_data = pd.read_csv("ccao_train.csv")
test_data = pd.read_csv("ccao_test.csv")

As a good sanity check, we should at least verify that the data shape matches the description.

In [3]:
# Edit these sanity checks to match the CCAO dataset.

# 2000 observations and 82 features in training data
assert training_data.shape == (2000, 82)
# 930 observations and 81 features in test data
assert test_data.shape == (930, 81)
# SalePrice is hidden in the test data
assert 'SalePrice' not in test_data.columns.values
# Every other column in the test data should be in the training data
assert len(np.intersect1d(test_data.columns.values, 
                          training_data.columns.values)) == 81

### Data Discovery

Prior to the open data initiative and even prior to the assessment modeling initiative, Cook County’s assessor office received much of their data for assessments from their relationships with [“local elected officials, community leaders, real estate professionals and other citizens knowledgeable about real estate in the area.”](https://www.cookcountyassessor.com/about-cook-county-assessors-office) Because CCAO field inspectors cannot enter homes to gather data,  this information must be gathered through either curbside observations or real estate records.

You can read more about data collection in the CCAO’s [Residential Data Integrity Preliminary Report](https://gitlab.com/ccao-data-science---modeling/ccao_sf_cama_dev/-/blob/master/documentation/Preliminary%20Report%20on%20Data%20Integrity%20June%207,%202019.pdf).


## HCE: Question 0

How is the CCAO’s data collected? Discuss an obstacle to CCAO's data collection efforts and address how it may limit the CCAO's goals.

`TODO`: *Write your answer here*

## HCE: Question 1

Take a moment to assess the granularity of this dataset. What types of information are contained in a row?

`TODO`: *Write your answer here*

## HCE: Question 2

Given that this data compiled by the CCAO (as opposed to other assessor’s offices), find a column that would most likely only be collected in Cook County. What does this column represent?

*Hint:*  Look for a feature related to Chicago’s airports.

`TODO`: *Write your answer here*

## HCE: Question 3

Name a feature that isn't listed in this dataset but may be useful for predicting sales values. What insights could this feature provide? How might it increase or decrease a home’s sales value?

*Hint:*  If you’re stuck, refer to the “Single-Family Properties Appropriateness” section in the CCAO’s [Residential Data Integrity Preliminary Report](https://gitlab.com/ccao-data-science---modeling/ccao_sf_cama_dev/-/blob/master/documentation/Preliminary%20Report%20on%20Data%20Integrity%20June%207,%202019.pdf).

`TODO`: *Write your answer here*

Now that we’re more familiar with the information that the dataset contains, let’s focus on the features themselves. Many features are categorical, utilizing classification to describe particular aspects of a home. Some features are straightforward in their categories - the `Wall Material` column, for example, has values that identify whether a property’s external wall is built with wood, masonry, both wood and masonry, or stucco. Others, however, are not so clearly defined.


## HCE: Question 4

Let’s take a look at the `Site Desirability` column. What do the column’s values represent? Does the documentation provide sufficient guidelines as to how a property's `Site Desirability` is determined? Why or why not?

`TODO`: *Write your answer here*

Just by looking at the feature and its description, we can tell that `Site Desirability` is highly discretionary - that is, it relies on an individual’s interpretation. And in this context, the individual in question would be a real estate agent or assessor.

## HCE: Question 5

Beyond a home’s internal characteristics (such as number of rooms, bathrooms, etc.), describe a factor that might influence whether a home is desirable, and elaborate why. Think from the perspective of a real estate agent - i.e. what would an agent market to potential buyers and why?

*Hint:*  Consider writing about characteristics related to a home’s location and its proximity to other places.

`TODO`: *Write your answer here*

When examining your data, it’s important to consider who collects the data in order to understand the assumptions and perspectives built into it. Here, the idea of desirable properties is influenced by what the real estate industry - a source of expertise in home valuations - deems popular with its market. These human choices about value are shaped by the real estate agents' expertise acquired through their professional training and experience. From a seller's perspective, this expertise also makes the agent's valuations legitimate and authoritative in a way that's difficult to contest. As a result, a form of professional bias enters into the valuation process, and though this bias isn’t inherently bad, we’ll delve deeper into how it interacts with, and often reinforces, structural inequity.

To continue using `Site Desirability` as an example, let’s say that proximity to high-ranking schools adds to a home’s desirability. Because schools are largely funded by property taxes, high-ranking schools are typically located near homes in higher-income neighborhoods. Comparatively, homes in lower-income neighborhoods - with families predominantly from Hispanic and Black communities - would be deemed “less desirable” because they would not feed into high-ranking schools, further devaluing homes in lower-income neighborhoods. 

In this way, `Site Desirability` acts as a proxy for sensitive attributes, such as the racial distribution of a neighborhood. Although it does not explicitly name these attributes, `Site Desirability` ultimately encodes the broader socioeconomic and racial context of a home’s surrounding community. And because real estate agents affirm their classifications of `Site Desirability` based on their expertise, structural inequity is further perpetuated as this bias is embedded in the data used to generate housing valuation models.



## HCE: Question 6

Although it’s impossible to remove bias (especially given that data is often collected through a specific profession’s lens), it’s nonetheless important to be aware of how choosing dataset features both reflects and produces real-world contexts. 

Find another feature in the dataset that, similarly to Site Desirability, encodes bias through expertise, and support your reasoning.

`TODO`: *Write your answer here*

Understanding your dataset provides a lot of insight into how models might incorporate bias from the very beginning. A data scientist with this awareness would not only identify sources of bias, but also aim to intentionally address these biases in their data analyses, as well as the outputs and recommendations based on these analyses. As we progress through this homework, keep these perspectives on bias and expertise in mind.

# Part 1: Exploratory Data Analysis

In this section, we will make a series of exploratory visualizations and interpret them. This will allow us to familiarize ourselves with how particular columns within the data might influence a model’s predictions, as well as expose discrepancies in data collection.

Note that we will perform EDA on the **training data** so that information from the test data does not influence our modeling decisions.


### Construction Quality 

As you may have observed in Part 0, `Construction Quality` is a feature that may introduce existing inequities to a model as a result of its discretionary classifications. Creating visualizations as part of EDA will allow us to further examine these classifications.


## HCE: Question 7

Look at the feature `Construction Quality` referenced in the [codebook](https://datacatalog.cookcountyil.gov/Property-Taxation/Cook-County-Assessor-s-Residential-Sales-Data/5pge-nu6u). Make a visualization of this feature and, in one sentence, describe a concern you might have about using this feature to predict a home’s value and why. 

*Hint:*  Pay attention to how the variable is distributed.

`TODO`: *Write your answer here*

As your work in Question 7 demonstrates, visualizations can directly reveal the underlying subtext of particular variables. Beyond the usefulness of the data itself, visualizing `Construction Quality` sheds light on deeper implications. Observe how average-rated houses are established as an overwhelming norm, which thus draws additional scrutiny to houses that are rated “Deluxe” and “Poor". Like `Site Desirability`, there is no additional context for these classifications, and the expertise involved in categorizing these properties potentially encodes real-life inequities that must be acknowledged.

### Sale Price
We begin by examining a [raincloud plot](https://micahallen.org/2018/03/15/introducing-raincloud-plots/amp/?__twitter_impression=true) (a combination of a KDE, a histogram, a strip plot, and a box plot) of our target variable `SalePrice`.  At the same time, we also take a look at some descriptive statistics of this variable.

In [5]:
fig, axs = plt.subplots(nrows=2)

sns.distplot(
    training_data['SalePrice'], 
    ax=axs[0]
)
sns.stripplot(
    training_data['SalePrice'], 
    jitter=0.4, 
    size=3,
    ax=axs[1],
    alpha=0.3
)
sns.boxplot(
    training_data['SalePrice'],
    width=0.3, 
    ax=axs[1],
    showfliers=False,
)

# Align axes
spacer = np.max(training_data['SalePrice']) * 0.05
xmin = np.min(training_data['SalePrice']) - spacer
xmax = np.max(training_data['SalePrice']) + spacer
axs[0].set_xlim((xmin, xmax))
axs[1].set_xlim((xmin, xmax))

# Remove some axis text
axs[0].xaxis.set_visible(False)
axs[0].yaxis.set_visible(False)
axs[1].yaxis.set_visible(False)

# Put the two plots together
plt.subplots_adjust(hspace=0)

# Adjust boxplot fill to be white
axs[1].artists[0].set_facecolor('white')

In [6]:
training_data['SalePrice'].describe()

## Question 1  <a name="q1"></a>
To check your understanding of the graph and summary statistics above, answer the following `True` or `False` questions:

1. The distribution of `SalePrice` in the training set is left-skew.
1. The mean of `SalePrice` in the training set is greater than the median.
1. At least 25% of the houses in the training set sold for more than \$200,000.00.

*The provided tests for this question do not confirm that you have answered correctly; only that you have assigned each variable to `True` or `False`.*

<!--
BEGIN QUESTION
name: q1
points: 3
-->

In [7]:
# These should be True or False
q1statement1 = ...
q1statement2 = ...
q1statement3 = ...

In [ ]:
ok.grade("q1");

### SalePrice vs Gr_Liv_Area

Next, we visualize the association between `SalePrice` and `Gr_Liv_Area`.  The `codebook.txt` file tells us that `Gr_Liv_Area` measures "above grade (ground) living area square feet."

This variable represents the square footage of the house excluding anything underground.  Some additional research (into real estate conventions) reveals that this value also excludes the garage space.

In [12]:
sns.jointplot(
    x='Gr_Liv_Area', 
    y='SalePrice', 
    data=training_data,
    stat_func=None,
    kind="reg",
    ratio=4,
    space=0,
    scatter_kws={
        's': 3,
        'alpha': 0.25
    },
    line_kws={
        'color': 'black'
    }
);

There's certainly an association, and perhaps it's linear, but the spread is wider at larger values of both variables.  Also, there are two particularly suspicious houses above 5000 square feet that look too inexpensive for their size.

## Question 2 <a name="q2"></a>
What are the Parcel Indentification Numbers for the two houses with `Gr_Liv_Area` greater than 5000 sqft?

*The provided tests for this question do not confirm that you have answered correctly; only that you have assigned `q2house1` and `q2house2` to two integers that are in the range of PID values.*

<!--
BEGIN QUESTION
name: q2
points: 2
-->

In [13]:
# Hint: You can answer this question in one line
...

In [ ]:
ok.grade("q2");

## Question 3 <a name="q3"></a>

The codebook tells us how to manually inspect the houses using an online database called Beacon. These two houses are true outliers in this data set: they aren't the same time of entity as the rest. They were partial sales, priced far below market value. If you would like to inspect the valuations, follow the directions at the bottom of the codebook to access Beacon and look up houses by PID.

For this assignment, we will remove these outliers from the data. Write a function `remove_outliers` that removes outliers from a data set based off a threshold value of a variable.  For example, `remove_outliers(training_data, 'Gr_Liv_Area', upper=5000)` should return a data frame with only observations that satisfy `Gr_Liv_Area` less than 5000.

*The provided tests check that training_data was updated correctly, so that future analyses are not corrupted by a mistake. However, the provided tests do not check that you have implemented remove_outliers correctly so that it works with any data, variable, lower, and upper bound.*

<!--
BEGIN QUESTION
name: q3
points: 1
-->

In [20]:
def remove_outliers(data, variable, lower=-np.inf, upper=np.inf):
    """
    Input:
      data (data frame): the table to be filtered
      variable (string): the column with numerical outliers
      lower (numeric): observations with values lower than or equal to this will be removed
      upper (numeric): observations with values higher than or equal to this will be removed
    
    Output:
      a winsorized data frame with outliers removed
      
    Note: This function should not change mutate the contents of data.
    """  
    ...

training_data = remove_outliers(training_data, 'Gr_Liv_Area', upper=5000)

In [ ]:
ok.grade("q3");

# Part 2: Feature Engineering

In this section, we will create a new feature out of existing ones through a simple data transformation. The goal is to implement broader domain knowledge we have about the data in order to aid our calculations. 

As an example, we can look to an instance of feature engineering in the CCAO’s dataset, `Building Square Feet`. Based on real estate norms, home appraisals can only consider the square footage of living space, which means that garage and basement sizes cannot be included in a home’s total square footage. This is reflected in the CCAO’s dataset: The description for the feature `Garage Area 1` mentions that if Garage 1 is physically included within the building’s area, the area of the garage must be subtracted from the building’s total square footage, `Building Square Feet`. 


## HCE: Question 8

Given that this data was used by the Cook County Assessor's Office to generate the model that they employ today, name two columns that have likely been added by the data scientists. Why do you think these transformations were created?

*Hint:* Look for columns that have been affected by mathematical transformations.


`TODO`: *Write your answer here*

### Bathrooms

Let's create a groundbreaking new feature. Due to recent advances in Universal WC Enumeration Theory, we now know that Total Bathrooms can be calculated as:

$$ \text{TotalBathrooms}=(\text{BsmtFullBath} + \text{FullBath}) + \dfrac{1}{2}(\text{BsmtHalfBath} + \text{HalfBath})$$

The actual proof is beyond the scope of this class, but we will use the result in our model.

## Question 4 <a name="q4"></a>

Write a function `add_total_bathrooms(data)` that returns a copy of `data` with an additional column called `TotalBathrooms` computed by the formula above.  **Treat missing values as zeros**.  Remember that you can make use of vectorized code here; you shouldn't need any `for` statements. 

*The provided tests check that you answered correctly, so that future analyses are not corrupted by a mistake.*

<!--
BEGIN QUESTION
name: q4
points: 1
-->

In [25]:
def add_total_bathrooms(data):
    """
    Input:
      data (data frame): a data frame containing at least 4 numeric columns 
            Bsmt_Full_Bath, Full_Bath, Bsmt_Half_Bath, and Half_Bath
    """
    with_bathrooms = data.copy()
    bath_vars = ['Bsmt_Full_Bath', 'Full_Bath', 'Bsmt_Half_Bath', 'Half_Bath']
    weights = pd.Series([1, 1, 0.5, 0.5], index=bath_vars)
    ...
    return with_bathrooms

training_data = add_total_bathrooms(training_data)

In [ ]:
ok.grade("q4");

## Question 5 <a name="q5"></a>

Create a visualization that clearly and succintly shows that `TotalBathrooms` is associated with `SalePrice`. Your visualization should avoid overplotting.

<!--
BEGIN QUESTION
name: q5
points: 2
manual: True
format: image
-->
<!-- EXPORT TO PDF format:image -->

In [30]:
...

# Part 3: Modeling

We've reached the point where we can specify a model. But first, we will load a fresh copy of the data, just in case our code above produced any undesired side-effects. Run the cell below to store a fresh copy of the data from `ccao_train.csv` in a dataframe named `full_data`. We will also store the number of rows in `full_data` in the variable `full_data_len`.

In [31]:
# Load a fresh copy of the data and get its length
full_data = pd.read_csv("ccao_train.csv")
full_data_len = len(full_data)
full_data.head()

## Question 6 <a name="q6"></a>

Now, let's split the data set into a training set and test set. We will use the training set to fit our model's parameters, and we will use the test set to estimate how well our model will perform on unseen data drawn from the same distribution. If we used all the data to fit our model, we would not have a way to estimate model performance on unseen data.

"Don't we already have a test set in `ccao_test.csv`?" you might wonder. The sale prices for `ccao_test.csv` aren't provided, so we're constructing our own test set for which we know the outputs.

In the cell below, split the data in `full_data` into two DataFrames named `train` and `test`. Let `train` contain 80% of the data, and let `test` contain the remaining 20% of the data. 

To do this, first create two NumPy arrays named `train_indices` and `test_indices`. `train_indices` should contain a *random* 80% of the indices in `full_data`, and `test_indices` should contain the remaining 20% of the indices. Then, use these arrays to index into `full_data` to create your final `train` and `test` DataFrames.

*The provided tests check that you not only answered correctly, but ended up with the exact same train/test split as our reference implementation. Later testing is easier this way.*

<!--
BEGIN QUESTION
name: q6
points: 2
-->

In [32]:
# This makes the train-test split in this section reproducible across different runs 
# of the notebook. You do not need this line to run train_test_split in general
np.random.seed(1337)
shuffled_indices = np.random.permutation(full_data_len)

# Set train_indices to the first 80% of shuffled_indices and and test_indices to the rest.
train_indices = ...
test_indices = ...

# Create train and test` by indexing into `full_data` using 
# `train_indices` and `test_indices`
train = ...
test = ...

In [ ]:
ok.grade("q6");

### Reusable Pipeline

Throughout this assignment, you should notice that your data flows through a single processing pipeline several times. From a software engineering perspective, it's ideal to define functions and methods that can apply the pipeline to any dataset. We will now encapsulate our entire pipeline into a single function `process_data_gm`. gm is shorthand for "guided model". 

Additionally, creating reproducible work is an important part of data science. When you leave a project, readability and transferability of your code is necessary for future work on the project by other data scientists. Therefore, it’s better to use abstract identifiers to navigate code. For example, in this notebook, we refer to columns by their names - that way, if a new dataset comes in with different column indices, we’ll be able to know which columns to target when adapting code.

In the CCAO's case, for example, they felt that making their model into a reproducible and easily reusable pipeline was an essential part of their mission as a public office. In this case, this allows citizens and other entities with sufficient technical skills to understand and even critique their work openly.


In [39]:
def select_columns(data, *columns):
    """Select only columns passed as arguments."""
    return data.loc[:, columns]

def process_data_gm(data):
    """Process the data for a guided model."""
    data = remove_outliers(data, 'Gr_Liv_Area', upper=5000)
    
    # Transform Data, Select Features
    data = add_total_bathrooms(data)
    data = select_columns(data, 
                          'SalePrice', 
                          'Gr_Liv_Area', 
                          'Garage_Area',
                          'TotalBathrooms',
                         )
    
    # Return predictors and response variables separately
    X = data.drop(['SalePrice'], axis = 1)
    y = data.loc[:, 'SalePrice']
    
    return X, y

Now, we can use `process_data_gm` to clean our data, select features, and add our `TotalBathrooms` feature all in one step! This function also splits our data into `X`, a matrix of features, and `y`, a vector of sale prices. 

Run the cell below to feed our training and test data through the pipeline, generating `X_train`, `y_train`, `X_test`, and `y_test`.

In [40]:
# Pre-process our training and test data in exactly the same way
# Our functions make this very easy!
X_train, y_train = process_data_gm(train)
X_test, y_test = process_data_gm(test)

### Fitting Our First Model

We are finally going to fit a model!  The model we will fit can be written as follows:

$$\text{SalePrice} = \theta_0 + \theta_1 \cdot \text{Gr_Liv_Area} + \theta_2 \cdot \text{Garage_Area} + \theta_3 \cdot \text{TotalBathrooms}$$

In vector notation, the same equation would be written:

$$y = \vec\theta \cdot \vec{x}$$

where $y$ is the SalePrice, $\vec\theta$ is a vector of all fitted weights, and $\vec{x}$ contains a 1 for the bias followed by each of the feature values.

**Note:** Notice that all of our variables are continuous, except for `TotalBathrooms`, which takes on discrete ordered values (0, 0.5, 1, 1.5, ...). In this homework, we'll treat `TotalBathrooms` as a continuous quantitative variable in our model, but this might not be the best choice. The next homework may revisit the issue.

## Question 7a <a name="q7a"></a>

We will use a [`sklearn.linear_model.LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) object as our linear model. In the cell below, create a `LinearRegression` object and name it `linear_model`.

**Hint:** See the `fit_intercept` parameter and make sure it is set appropriately. The intercept of our model corresponds to $\theta_0$ in the equation above.

*The provided tests check that you answered correctly, so that future analyses are not corrupted by a mistake.*

<!--
BEGIN QUESTION
name: q7a
points: 1
-->

In [41]:
from sklearn import linear_model as lm

linear_model = ...

In [ ]:
ok.grade("q7a");

## Question 7b <a name="q7b"></a>

Now, remove the commenting and fill in the ellipses `...` below with `X_train`, `y_train`, `X_test`, or `y_test`.

With the ellipses filled in correctly, the code below should fit our linear model to the training data and generate the predicted sale prices for both the training and test datasets.

*The provided tests check that you answered correctly, so that future analyses are not corrupted by a mistake.*

<!--
BEGIN QUESTION
name: q7b
points: 2
-->

In [44]:
# Uncomment the lines below and fill in the ... with X_train, y_train, X_test, or y_test.
# linear_model.fit(..., ...)
# y_fitted = linear_model.predict(...)
# y_predicted = linear_model.predict(...)

In [ ]:
ok.grade("q7b");

## Question 8a <a name="q8a"></a>

Is our linear model any good at predicting house prices? Let's measure the quality of our model by calculating the Root-Mean-Square Error (RMSE) between our predicted house prices and the true prices stored in `SalePrice`.

$$\text{RMSE} = \sqrt{\dfrac{\sum_{\text{houses in test set}}(\text{actual price of house} - \text{predicted price of house})^2}{\text{# of houses in data set}}}$$

In the cell below, write a function named `rmse` that calculates the RMSE of a model.

**Hint:** Make sure you are taking advantage of vectorized code. This question can be answered without any `for` statements.

*The provided tests check that you answered correctly, so that future analyses are not corrupted by a mistake.*

<!--
BEGIN QUESTION
name: q8a
points: 1
-->

In [47]:
def rmse(actual, predicted):
    """
    Calculates RMSE from actual and predicted values
    Input:
      actual (1D array): vector of actual values
      predicted (1D array): vector of predicted/fitted values
    Output:
      a float, the root-mean square error
    """
    ...

In [ ]:
ok.grade("q8a");

## Question 8b <a name="q8b"></a>

Now use your `rmse` function to calculate the training error and test error in the cell below.

*The provided tests for this question do not confirm that you have answered correctly; only that you have assigned each variable to a non-negative number.*

<!--
BEGIN QUESTION
name: q8b
points: 1
-->

In [50]:
training_error = ...
test_error = ...
(training_error, test_error)

In [ ]:
ok.grade("q8b");

## Question 8c <a name="q8c"></a>

How much does including `TotalBathrooms` as a predictor reduce the RMSE of the model on the test set? That is, what's the difference between the RSME of a model that only includes `Gr_Liv_Area` and `Garage_Area` versus one that includes all three predictors?

*The provided tests for this question do not confirm that you have answered correctly; only that you have assigned the answer variable to a non-negative number.*

<!--
BEGIN QUESTION
name: q8c
points: 2
-->

In [55]:
test_error_difference = test_error_no_bath - test_error
test_error_difference

In [ ]:
ok.grade("q8c");

### Residual Plots

One way of understanding the performance (and appropriateness) of a model is through a residual plot. Run the cell below to plot the actual sale prices against the residuals of the model for the test data.

In [58]:
residuals = y_test - y_predicted
ax = sns.regplot(y_test, residuals)
ax.set_xlabel('Sale Price (Test Data)')
ax.set_ylabel('Residuals (Actual Price - Predicted Price)')
ax.set_title("Residuals vs. Sale Price on Test Data");

Ideally, we would see a horizontal line of points at 0 (perfect prediction!). The next best thing would be a homogenous set of points centered at 0. 

But alas, our simple model is probably too simple. The most expensive homes are systematically more expensive than our prediction. 

## Question 8d <a name="q8c"></a>

What changes could you make to your linear model to improve its accuracy and lower the test error? Suggest at least two things you could try in the cell below, and carefully explain how each change could potentially improve your model's accuracy.

<!--
BEGIN QUESTION
name: q8d
points: 2
manual: True
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

## HCE: Question 9 

In the context of assessing houses, what does error mean for an individual homeowner? How does it affect them in terms of property taxes? Do these effects change your tolerance for error in modeling?


`TODO`: *Write your answer here*

Congratulations! You’ve built a simple home assessment model. With this simple model, it's clear that the main variables in the model are the Living Area, Garage Area, and TotalBathrooms. However, as you might imagine, the CCAO's model is much more complex. In the next homework, we'll discuss the CCAO's model with more depth as we continue to improve the simple model we built here.

In the meantime, consider the implications of your work so far: We know that housing assessments affect property taxes. Fairness in the CCAO’s work, then, is accurately assessing the value of homeowners’ properties, where accuracy is a reflection of whether a homeowner pays taxes comparable to homeowners with similar houses. These assessments - and even what’s considered accurate - are a product of choices made at every level of the assessment process, from data collection to modeling. And the humans who make these choices - real estate agents, data scientists, county assessors, and, in this homework and the next, you - must understand the inputs to and consider the impacts of these decisions.

So when we conceptualize accurate assessments, we must think beyond eliminating bias. Bias is impossible to eliminate, given the indispensability of human involvement and historical/institutional contexts. We must instead consider the long-term, real-world consequences for individuals and communities in specific local and historical contexts. Only in this way can we aspire towards making fair and equitable assessment possible.

# Submit
Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output.
**Please save before submitting!**

<!-- EXPECT 2 EXPORTED QUESTIONS -->

In [ ]:
# Save your notebook first, then run this cell to submit.
import jassign.to_pdf
jassign.to_pdf.generate_pdf('hw5.ipynb', 'hw5.pdf')
ok.submit()